In [1]:
import nltk   
nltk.download('vader_lexicon') #the nltk uses the pretrained model called Vader which has to be downloaded.
# Vader uses bag of words and looks at each word and scores it individually without considering the relationship between the words or context
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
plt.style.use('ggplot')
import seaborn as sns
import emoji
import demoji
import spacy

from nltk.sentiment import SentimentIntensityAnalyzer
from transformers import pipeline
import os


[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /Users/giu/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


In [2]:
Dataset_name = 'TechLabsDataset.csv' 
DataDictionary_name = 'TechLabsDataset_Dictionary.csv'

Dataset = pd.read_csv(os.path.join('./Data/',Dataset_name), index_col = 0)
DataDictionary = pd.read_csv(os.path.join('./Data/',DataDictionary_name), index_col = 0)
Dataset.head()

,name,type_company,website,lat,lon,city,avg_stars_num,n_reviews_num,encoded_user,local_guide,clean_other_review_num,published_date,today_date,stars_num,review_EN,original_lang
0,Caretrex warehousing & logistics,Logistics service,http://www.caretrex.com/,51.593721,5.073492,Tilburg,5.0,2.0,HHpDM,0.0,4.0,2 years ago,2022-09-24,5.0,NaN,NaN
1,Caretrex warehousing & logistics,Logistics service,http://www.caretrex.com/,51.593721,5.073492,Tilburg,5.0,2.0,dYFWx,0.0,1.0,5 years ago,2022-09-24,5.0,NaN,NaN
2,FEFA Logistics,Trucking company,http://www.fefalogistics.nl/,51.480074,5.446764,Eindhoven,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Dekkers Transport Holland,Trucking company,http://www.dekkerstransport.nl/,51.597249,5.027990,Tilburg,4.6,53.0,Pp1)c,0.0,5.0,a year ago,2022-09-24,5.0,Best employer ever. Worked there for approxima...,ENG
4,Dekkers Transport Holland,Trucking company,http://www.dekkerstransport.nl/,51.597249,5.027990,Tilburg,4.6,53.0,*nk1l,0.0,2.0,a year ago,2022-09-24,5.0,better place with very good people everything ...,ENG


In [3]:
review_df = pd.DataFrame(Dataset[~Dataset['review_EN'].isnull()]['review_EN'])
review_df.head()

,review_EN
3,Best employer ever. Worked there for approxima...
4,better place with very good people everything ...
5,Nice
6,Good firm. The staff are very welcoming and f...
7,"Top company, and coffee, so recommended\n\n"


In [4]:
# find emo
def find_emo(text):
    # find emoji and add to list
    all_emoji = demoji.findall(text)
    emo = ''
    if len(all_emoji)>0:
        for i,j in enumerate(all_emoji):
            # list emo
            emo = emo+','+all_emoji[j]
    return emo


# replace emo with empty text
def replace_emo(text):
    # find emo and replce with empty text
    all_emoji = demoji.findall(text)

    if len(all_emoji)>0:
        for i,j in enumerate(all_emoji):
            text = text.replace(j,'')
    return text

In [21]:
# add review with no emo
# replace emo with empty text
def replace_emo(text):
    # find emo and replce with empty text
    all_emoji = demoji.findall(text)

    if len(all_emoji)>0:
        for i,j in enumerate(all_emoji):
            text = text.replace(j,'')
    return text

M1_vader = SentimentIntensityAnalyzer()# Vader Model
review_df['review_no_emo'] = review_df['review_EN'].apply(lambda x: replace_emo(x))
review_df['review_no_emo'] = review_df['review_no_emo'].apply(lambda x: x.replace('\n',' '))
review_df[['neg','neu','pos','compound']] = review_df['review_no_emo'].apply(lambda x: pd.Series(M1_vader.polarity_scores(x)))

review_df[['neg','neu','pos','compound']]

In [35]:
review_df[['neg','neu','pos','compound']]

,neg,neu,pos,compound
3,0.031,0.821,0.147,0.6992
4,0.000,0.380,0.620,0.8687
5,0.000,0.000,1.000,0.4215
6,0.000,0.347,0.653,0.9512
7,0.000,0.490,0.510,0.4882
...,...,...,...,...
8068,0.233,0.666,0.102,-0.5783
8069,0.110,0.890,0.000,-0.4767
8070,0.165,0.711,0.124,-0.1756
8071,0.000,1.000,0.000,0.0000


In [34]:
from transformers import pipeline
# Too slow
# M2_sentiment_pipeline = pipeline("sentiment-analysis")
# review_df['review_no_emo'].apply(lambda x: pd.Series(M2_sentiment_pipeline(x)[0]))


# https://huggingface.co/Souvikcmsa/BERT_sentiment_analysis
M3_classifier = pipeline("text-classification", model = "Souvikcmsa/BERT_sentiment_analysis")
review_df['review_no_emo'].apply(lambda x: pd.Series(M3_classifier(x)[0]))

,label,score
3,neutral,0.675914
4,positive,0.972374
5,positive,0.971999
6,positive,0.984962
7,positive,0.829312
...,...,...
8068,neutral,0.527962
8069,negative,0.928342
8070,positive,0.931631
8071,neutral,0.958836


In [ ]:
# https://huggingface.co/Souvikcmsa/Roberta_Sentiment_Analysis
# works with Emoij

from transformers import AutoModelForSequenceClassification, AutoTokenizer

model = AutoModelForSequenceClassification.from_pretrained("Souvikcmsa/autotrain-sentimentAnalysis_By_Souvik-762623422", use_auth_token=True)

tokenizer = AutoTokenizer.from_pretrained("Souvikcmsa/autotrain-sentimentAnalysis_By_Souvik-762623422", use_auth_token=True)

inputs = tokenizer("I love AutoTrain", return_tensors="pt")

outputs = model(**inputs)


In [ ]:
# https://huggingface.co/bgoel4132/twitter-sentiment - NO
# works with emoij
# from transformers import AutoModelForSequenceClassification, AutoTokenizer

# model = AutoModelForSequenceClassification.from_pretrained("bgoel4132/autonlp-twitter-sentiment-35868888", use_auth_token=True)

# tokenizer = AutoTokenizer.from_pretrained("bgoel4132/autonlp-twitter-sentiment-35868888", use_auth_token=True)

# inputs = tokenizer("I love AutoNLP", return_tensors="pt")

# outputs = model(**inputs)

In [ ]:
# https://huggingface.co/amansolanki/autonlp-Tweet-Sentiment-Extraction-20114061
from transformers import AutoModelForSequenceClassification, AutoTokenizer

model = AutoModelForSequenceClassification.from_pretrained("amansolanki/autonlp-Tweet-Sentiment-Extraction-20114061", use_auth_token=True)

tokenizer = AutoTokenizer.from_pretrained("amansolanki/autonlp-Tweet-Sentiment-Extraction-20114061", use_auth_token=True)

inputs = tokenizer("I love AutoNLP", return_tensors="pt")

outputs = model(**inputs)

In [33]:
classifier = pipeline("text-classification", model = "Souvikcmsa/BERT_sentiment_analysis")
pd.Series(classifier("I loved Star Wars not so much!")[0])# Positive

label    neutral
score    0.48146
dtype: object